In [1]:
#loading the libraries
library(data.table)
library(enrichR)
library(ggplot2)

Welcome to enrichR
Checking connection ... 

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is Live!

WormEnrichr ... 
Connection is Live!

YeastEnrichr ... 
Connection is Live!

FishEnrichr ... 
Connection is Live!

OxEnrichr ... 
Connection is Live!



In [2]:
#general settings for the plots
theme_set(theme_bw())
theme_update(axis.text = element_text(color="black"),
              panel.grid.minor = element_blank(), 
             text = element_text(size=12))

In [3]:
#general pdf setting
pdf.options(useDingbats=FALSE)


In [4]:
#plotting helper functions

rotate_labels <- function(angle = 60,vjust = 1,hjust = 1)
{return(theme(axis.text.x = element_text(angle = angle, vjust = vjust,hjust = hjust)))}

In [5]:
#setting the working directory
setwd("/home//mmokhtari/INF_analysis/enrichment plot/")

In [6]:
de=fread("DE_genes_donor0_vs_donor_1.csv")

In [7]:
#filtering the DEs
de_filt=de[abs(logfoldchanges)>1.5&pvals_adj<0.05]
de_filt[,direction:=ifelse(logfoldchanges>0,"up","down"),]


In [8]:
#running the enrichment analysis on the data

enrich=de_filt[,as.data.table(enrichr(names,databases="GO_Biological_Process_2018")),by=c("Cluster","direction")]
enrich[,Cluster:=gsub("-donor","",Cluster),]
setnames(enrich,names(enrich),gsub("GO_Biological_Process_2018\\.","",names(enrich)))



Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  Querying GO_Biological_Process_2018... Done.
Parsing results... Done.
Uploading data to Enrichr... Done.
  

In [9]:
#subsetting the upregulated enrichment terms
sel=c("up",0.005)
#sel=c("down",0.05)

enrich_up=enrich[direction==sel[1]&Adjusted.P.value<as.numeric(sel[2])]
enrich_up[,GO:=gsub(".* |\\(|\\)|\\:","",Term),]
enrich_up[,GO_orig:=gsub(".* |\\(|\\)","",Term),]
enrich_up[,N:=.N,by="GO"]


In [10]:
#sorting the terms for the plot
enrich_up[,Cluster:=factor(Cluster,c("B","CD4+_T","CD8+_T","NK","CD16+_Monocyte","CD14+_Monocyte","cDC","Platelet")),]
enrich_up[,cluster_sum:=sum(-as.numeric(Cluster)),by=Term]
enrich_up[,cluster_min:=-min(as.numeric(Cluster)),by=Term]
enrich_up[,pval_min:=-min(Adjusted.P.value),by=Term]
enrich_up[,or_max:=max(Odds.Ratio),by=Term]
enrich_up[,Term_f:=factor(Term,levels=unique(Term[order(N,cluster_sum,cluster_min,or_max,pval_min)])),]


write.table(enrich_up,paste0("GO_terms_",sel[1],".tsv"),sep="\t", row.names = F)


In [11]:
#visualisation of the enrichment terms

pdf(paste0("GO_",sel[1],"_",sel[2],".pdf"),width = 13,height = 15)
ggplot(enrich_up,aes(y=Term_f,x=Cluster,
                     size=Odds.Ratio,
                     colour=-log10(Adjusted.P.value)))+geom_point()+
  rotate_labels()+scale_color_gradient(high="red",low="blue")+
  scale_size_continuous(range=c(1,4))
dev.off()  



png 
  2